In [3]:
import math
import time
import numpy as np
import json
import glob
import warnings
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)


import datetime



C:\Users\enriq\AppData\Roaming\Python\Python39\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\enriq\AppData\Local\Temp\ipykernel_24980\727625543.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.

# Import der Daten

In [4]:
df = pd.read_pickle('dfipynb.pickle')
dftmp=df

# Datumszeile erstellen

In [5]:
def anpassung_zeit(df):
    df = df[df['Querschnitt'].notna()]
    df = df[df['Datum'].notna()]
    df.HHMM = df.HHMM / 100
    df['HHMM']=df['HHMM'].replace(24.0,23.59)
    df.head(100)
    df['Datum und Zeit'] = df["Datum"].astype(str).str[:10]+ ' ' +df.HHMM.astype(str)
    df['Datumszeit'] = pd.to_datetime(df['Datum und Zeit'], format='%Y-%m-%d %H.%M', errors= "coerce")
    return df
df=anpassung_zeit(df)

In [28]:
df

,Zst,Datum,HHMM,Richtung,-,Klasse,CS 1,CS 2,CS 3,CS 4,CS 5,CS 6,CS 7,CS 8,CS 9,CS10,Day Helper,Month Helper,Querschnitt,Wochenende/Feirtag,Datum und Zeit,Datumszeit
1,5401,2022-01-01 00:00:00,1.00,1.0,0.0,2.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,20.0,x,2022-01-01 1.0,2022-01-01 01:00:00
3,5401,2022-01-01 00:00:00,1.00,2.0,0.0,2.0,1.0,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,23.0,x,2022-01-01 1.0,2022-01-01 01:00:00
5,5401,2022-01-01 00:00:00,2.00,1.0,0.0,2.0,1.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,32.0,x,2022-01-01 2.0,2022-01-01 02:00:00
7,5401,2022-01-01 00:00:00,2.00,2.0,0.0,2.0,1.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,38.0,x,2022-01-01 2.0,2022-01-01 02:00:00
9,5401,2022-01-01 00:00:00,3.00,1.0,0.0,2.0,1.0,1.0,23.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,26.0,x,2022-01-01 3.0,2022-01-01 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3342363,9005.0,2021-12-31 00:00:00,22.00,2.0,0.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,4.0,NaN,2021-12-31 22.0,2021-12-31 22:00:00
3342365,9005.0,2021-12-31 00:00:00,23.00,1.0,0.0,2.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,9.0,NaN,2021-12-31 23.0,2021-12-31 23:00:00
3342367,9005.0,2021-12-31 00:00:00,23.00,2.0,0.0,2.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,4.0,NaN,2021-12-31 23.0,2021-12-31 23:00:00
3342369,9005.0,2021-12-31 00:00:00,23.59,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,3.0,NaN,2021-12-31 23.59,2021-12-31 23:59:00


df1.HHMM = df1.HHMM / 100
df1['HHMM']=df1['HHMM'].replace(24.0,0.0)
df1.head(100)
df1['Datum und Zeit'] = df1["Datum"].astype(str)+ ' ' +df1.HHMM.astype(str)
df1['Datumszeit'] = pd.to_datetime(df1['Datum und Zeit'], format='%Y-%m-%d %H.%M')

In [33]:
import pandas as pd
import pandasql as psql



def addAggregationsForEachDay(df):
    carsPerHour = "SELECT Zst, Datumszeit, sum(Querschnitt) as anzahlZurStunde FROM df GROUP BY Zst, Datumszeit"
    df = psql.sqldf(carsPerHour)

    zstMeasruesPerDay = "SELECT Zst, cast(Datumszeit as date) as Datum, sum(anzahlZurStunde) as amountOnThisDay, sum(anzahlZurStunde)/24 as avgPerHourOnThisDay, min(anzahlZurStunde) as lowestAmountOnThisDay, max(anzahlZurStunde) as highestAmountOnThisDay " \
                        "FROM df " \
                        "GROUP BY Zst, Datum"
    df = psql.sqldf(zstMeasruesPerDay)
    return df

addAggregationsForEachDay(df)



InterfaceError: (sqlite3.InterfaceError) Error binding parameter 1 - probably unsupported type.
[SQL: INSERT INTO df ("Zst", "Datum", "HHMM", "Richtung", "-", "Klasse", "CS 1", "CS 2", "CS 3", "CS 4", "CS 5", "CS 6", "CS 7", "CS 8", "CS 9", "CS10", "Day Helper", "Month Helper", "Querschnitt", "Wochenende/Feirtag", "Datum und Zeit", "Datumszeit") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((5401, Timestamp('2022-01-01 00:00:00'), 1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 1.0, 20.0, 'x', '2022-01-01 1.0', '2022-01-01 01:00:00.000000'), (5401, Timestamp('2022-01-01 00:00:00'), 1.0, 2.0, 0.0, 2.0, 1.0, 1.0, 21.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 1.0, 23.0, 'x', '2022-01-01 1.0', '2022-01-01 01:00:00.000000'), (5401, Timestamp('2022-01-01 00:00:00'), 2.0, 1.0, 0.0, 2.0, 1.0, 0.0, 31.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 1.0, 32.0, 'x', '2022-01-01 2.0', '2022-01-01 02:00:00.000000'), (5401, Timestamp(

In [50]:
temp = df.head(1000)
sql1 =  "SELECT Zst, Datumszeit, sum(Querschnitt) as anzahlZurStunde FROM temp GROUP BY Zst, Datumszeit"
temp = psql.sqldf(sql1)

convertToDate= "Select cast(Datumszeit as date) as Datum from temp"
psql.sql(convertToDate)

sql2 = "SELECT Zst, cast(Datumszeit as date) as Datum, sum(anzahlZurStunde) as amountOnThisDay, sum(anzahlZurStunde)/24 as avgPerHourOnThisDay, min(anzahlZurStunde) as lowestAmountOnThisDay, max(anzahlZurStunde) as highestAmountOnThisDay " \
                        "FROM temp " \
                        "GROUP BY Zst, cast(Datumszeit as date)"

temp = psql.sqldf(sql2)

AttributeError: module 'pandasql' has no attribute 'sql'

In [55]:
temp = df.head(1000)
temp = psql.sqldf(sql1)
temp

,Zst,Datumszeit,anzahlZurStunde
0,5401,2022-01-01 01:00:00.000000,43.0
1,5401,2022-01-01 02:00:00.000000,70.0
2,5401,2022-01-01 03:00:00.000000,62.0
3,5401,2022-01-01 04:00:00.000000,41.0
4,5401,2022-01-01 05:00:00.000000,22.0
...,...,...,...
495,5401,2022-01-21 16:00:00.000000,373.0
496,5401,2022-01-21 17:00:00.000000,489.0
497,5401,2022-01-21 18:00:00.000000,576.0
498,5401,2022-01-21 19:00:00.000000,376.0


In [63]:
tmp = df
index = pd.DatetimeIndex(tmp['Datumszeit'])
tmp = tmp.iloc[index.indexer_between_time('06:30', '22:30')]
# group by measurepoint and date, then apply aggregations
result = tmp.groupby(['Zst', 'Datumszeit']).agg({'Querschnitt': ['sum']}).reset_index()
result["Datum"] =  result["Datumszeit"].dt.date
result.columns = [''.join(col).strip() for col in result.columns.values]
result = result.groupby(['Zst', 'Datum']).agg({'Querschnittsum': ['sum','min','max']}).reset_index()
result.columns = ['_'.join(col).strip() for col in result.columns.values]
result['diff'] = result['Querschnittsum_max']-result['Querschnittsum_min']
result['avg/h'] = result['Querschnittsum_sum']/17

In [64]:
result

,Zst_,Datum_,Querschnittsum_sum,Querschnittsum_min,Querschnittsum_max,diff,avg/h
0,5401.0,2021-01-04,11079.0,174.0,1044.0,870.0,651.705882
1,5401.0,2021-01-05,10533.0,144.0,1077.0,933.0,619.588235
2,5401.0,2021-01-06,7458.0,177.0,756.0,579.0,438.705882
3,5401.0,2021-01-07,11790.0,195.0,1107.0,912.0,693.529412
4,5401.0,2021-01-08,12453.0,240.0,1353.0,1113.0,732.529412
...,...,...,...,...,...,...,...
29479,9005.0,2021-12-27,933.0,14.0,111.0,97.0,54.882353
29480,9005.0,2021-12-28,810.0,12.0,97.0,85.0,47.647059
29481,9005.0,2021-12-29,634.0,13.0,74.0,61.0,37.294118
29482,9005.0,2021-12-30,822.0,17.0,89.0,72.0,48.352941


In [89]:
import plotly.express as px
fig = px.bar(result, x='Zst_', y='avg/h')

fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})
fig.update_xaxes(type='category')

fig.show()

In [90]:
import plotly.express as px


fig = px.bar(result, x='Zst_', y='diff')
fig.update_layout(barmode='group', xaxis={'categoryorder':'total descending'})

fig.update_xaxes(type='category')

fig.show()

In [83]:
import plotly.express as px
result['rel'] = result['diff']/result['avg/h']

fig = px.bar(result, x='Zst_', y='rel')

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.update_xaxes(type='category')

fig.show()

In [75]:
result

,Zst_,Datum_,Querschnittsum_sum,Querschnittsum_min,Querschnittsum_max,diff,avg/h
0,5401.0,2021-01-04,11079.0,174.0,1044.0,870.0,651.705882
1,5401.0,2021-01-05,10533.0,144.0,1077.0,933.0,619.588235
2,5401.0,2021-01-06,7458.0,177.0,756.0,579.0,438.705882
3,5401.0,2021-01-07,11790.0,195.0,1107.0,912.0,693.529412
4,5401.0,2021-01-08,12453.0,240.0,1353.0,1113.0,732.529412
...,...,...,...,...,...,...,...
29479,9005.0,2021-12-27,933.0,14.0,111.0,97.0,54.882353
29480,9005.0,2021-12-28,810.0,12.0,97.0,85.0,47.647059
29481,9005.0,2021-12-29,634.0,13.0,74.0,61.0,37.294118
29482,9005.0,2021-12-30,822.0,17.0,89.0,72.0,48.352941
